# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [2]:
df_receipt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104681 entries, 0 to 104680
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   sales_ymd       104681 non-null  int64 
 1   sales_epoch     104681 non-null  int64 
 2   store_cd        104681 non-null  object
 3   receipt_no      104681 non-null  int64 
 4   receipt_sub_no  104681 non-null  int64 
 5   customer_id     104681 non-null  object
 6   product_cd      104681 non-null  object
 7   quantity        104681 non-null  int64 
 8   amount          104681 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 7.2+ MB


In [3]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21971 entries, 0 to 21970
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   customer_id           21971 non-null  object
 1   customer_name         21971 non-null  object
 2   gender_cd             21971 non-null  int64 
 3   gender                21971 non-null  object
 4   birth_day             21971 non-null  object
 5   age                   21971 non-null  int64 
 6   postal_cd             21971 non-null  object
 7   address               21971 non-null  object
 8   application_store_cd  21971 non-null  object
 9   application_date      21971 non-null  int64 
 10  status_cd             21971 non-null  object
dtypes: int64(3), object(8)
memory usage: 1.8+ MB


In [4]:
# P-70 と同様の問題
# まず merge() をして行数を調べる
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')
df_tmp.head()

,customer_id,sales_ymd,application_date
0,CS006214000001,20181103,20150201
1,CS006214000001,20170509,20150201
2,CS006214000001,20170608,20150201
3,CS006214000001,20170608,20150201
4,CS006214000001,20181028,20150201


In [5]:
df_tmp.shape

(65682, 3)

In [6]:
# 重複を削除するメソッド drop_duplicate()
df_tmp = df_tmp.drop_duplicates()
df_tmp.shape

(32411, 3)

In [7]:
# strデータを日付データに変換
# 事前にstr型にキャストする必要あり
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str'))
df_tmp.head()

,customer_id,sales_ymd,application_date
0,CS006214000001,2018-11-03,2015-02-01
1,CS006214000001,2017-05-09,2015-02-01
2,CS006214000001,2017-06-08,2015-02-01
4,CS006214000001,2018-10-28,2015-02-01
7,CS006214000001,2019-09-08,2015-02-01


In [8]:
# lambda関数で計算
df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: 
                      relativedelta(x[0], x[1]).years * 12 + relativedelta(x[0], x[1]).months, axis=1)
df_tmp.sort_values('customer_id').head(10)

,customer_id,sales_ymd,application_date,elapsed_date
60376,CS001113000004,2019-03-08,2015-11-05,40
20158,CS001114000005,2019-07-31,2016-04-12,39
20156,CS001114000005,2018-05-03,2016-04-12,24
29140,CS001115000010,2019-04-05,2015-04-17,47
29141,CS001115000010,2018-07-01,2015-04-17,38
29142,CS001115000010,2017-12-28,2015-04-17,32
6526,CS001205000004,2019-06-25,2016-06-15,36
6521,CS001205000004,2019-03-12,2016-06-15,32
6518,CS001205000004,2018-08-21,2016-06-15,26
6520,CS001205000004,2017-09-14,2016-06-15,14


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [9]:
# P-70,71 と同様の問題
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str')) # 修正

# 月をlambda関数で計算
df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: 
                                                     relativedelta(x[0], x[1]).years, axis=1)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
3,CS006214000001,2017-06-08,2015-02-01,2
4,CS006214000001,2018-10-28,2015-02-01,3
5,CS006214000001,2018-10-28,2015-02-01,3
6,CS006214000001,2017-05-09,2015-02-01,2
7,CS006214000001,2019-09-08,2015-02-01,4
8,CS006214000001,2018-01-31,2015-02-01,2
9,CS006214000001,2017-07-05,2015-02-01,2


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [10]:
# P-70 ～ 72 と同様の問題
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')

df_tmp = df_tmp.drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str')) # 修正

# # 公式解答：UNIX のエポック秒に変換する方法
df_tmp['elapsed_date'] = (df_tmp['sales_ymd'].astype(np.int64) / 10**9) - (df_tmp['application_date'].astype(np.int64) / 10**9)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0
10,CS006214000001,2018-11-10,2015-02-01,119059200.0
12,CS006214000001,2019-04-10,2015-02-01,132105600.0
15,CS006214000001,2019-06-01,2015-02-01,136598400.0


In [11]:
# timestamp() を使う方法
# https://note.nkmk.me/python-unix-time-datetime/

# lambda 関数を使ってすべての行にapply() させる。
sales_ymd = df_tmp['sales_ymd'].apply(lambda x: x.timestamp()) 
application_date = df_tmp['application_date'].apply(lambda x: x.timestamp())

df_tmp['elapsed_date']  = sales_ymd - application_date
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0
10,CS006214000001,2018-11-10,2015-02-01,119059200.0
12,CS006214000001,2019-04-10,2015-02-01,132105600.0
15,CS006214000001,2019-06-01,2015-02-01,136598400.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [12]:
# 公式解答例のままです。
df_tmp = df_receipt[['customer_id', 'sales_ymd']]
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))

# これがわからなかった
# relativedelta() --> 日付の加算、減算
# https://qiita.com/xza/items/9618e25a8cb08c44cdb0
# 最初に読み込んだこのライブラリーを使っています。
# from dateutil.relativedelta import relativedelta

df_tmp['monday'] = df_tmp['sales_ymd'].apply(lambda x: x - relativedelta(days=x.weekday()))

df_tmp['elapsed_weekday'] = df_tmp['sales_ymd'] - df_tmp['monday']
df_tmp.head(10)

,customer_id,sales_ymd,monday,elapsed_weekday
0,CS006214000001,2018-11-03,2018-10-29,5 days
1,CS008415000097,2018-11-18,2018-11-12,6 days
2,CS028414000014,2017-07-12,2017-07-10,2 days
3,ZZ000000000000,2019-02-05,2019-02-04,1 days
4,CS025415000050,2018-08-21,2018-08-20,1 days
5,CS003515000195,2019-06-05,2019-06-03,2 days
6,CS024514000042,2018-12-05,2018-12-03,2 days
7,CS040415000178,2019-09-22,2019-09-16,6 days
8,ZZ000000000000,2017-05-04,2017-05-01,3 days
9,CS027514000015,2019-10-10,2019-10-07,3 days


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [13]:
# https://note.nkmk.me/python-pandas-sample/
df_customer.sample(frac=0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
11297,CS032414000003,藤村 みき,1,女性,1976-03-04,43,144-0056,東京都大田区西六郷**********,S13032,20150418,F-20101027-E
7654,CS028615000103,車 たまき,1,女性,1951-08-18,67,246-0022,神奈川県横浜市瀬谷区三ツ境**********,S14028,20141115,C-20100531-9
3238,CS008503000041,松山 隼士,0,男性,1964-12-18,54,201-0015,東京都狛江市猪方**********,S13008,20150311,0-00000000-0
19590,CS004513000378,向井 紗季,1,女性,1966-07-19,52,176-0024,東京都練馬区中村**********,S13004,20160601,2-20090608-1
2690,CS035613000125,山内 結子,1,女性,1950-02-10,69,158-0095,東京都世田谷区瀬田**********,S13035,20150712,0-00000000-0
19677,CS025213000030,青山 めぐみ,1,女性,1991-08-11,27,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20150112,0-00000000-0
2453,CS003602000058,宮原 米蔵,0,男性,1955-10-07,63,214-0012,神奈川県川崎市多摩区中野島**********,S13003,20160406,0-00000000-0
5524,CS023312000102,羽田 千夏,1,女性,1981-03-26,38,212-0054,神奈川県川崎市幸区小倉**********,S14023,20150407,0-00000000-0
11572,CS037613000063,安斎 璃子,1,女性,1956-06-02,62,136-0072,東京都江東区大島**********,S13037,20150316,0-00000000-0
5537,CS003413000586,竹中 ひろ子,1,女性,1970-04-16,48,201-0001,東京都狛江市西野川**********,S13003,20171017,0-00000000-0
